<a href="https://colab.research.google.com/github/YOUR_USERNAME/rudushi/blob/main/Rudushi_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Rudushi Fine-Tuning Notebook

**Create your custom Rudushi AI assistant!**

This notebook will fine-tune TinyLlama-1.1B on the Alpaca dataset to create your custom Rudushi model.

## ⏱️ Estimated Time: 30-90 minutes

## Step 1: Install Dependencies

In [ ]:
# Install required packages
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes datasets transformers huggingface_hub safetensors
!pip install -q ipywidgets

print("✅ All dependencies installed!")

## Step 2: Import Libraries

In [ ]:
import torch
from datasets import load_dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import os

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

## Step 3: Load Base Model (TinyLlama-1.1B)

In [ ]:
# Configuration
MODEL_NAME = "unsloth/tinyllama-bnb-4bit"
MAX_SEQ_LENGTH = 2048
DTYPE = None
LOAD_IN_4BIT = True

print("📦 Loading TinyLlama-1.1B model...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=DTYPE,
    load_in_4bit=LOAD_IN_4BIT,
)

print(f"✅ Model loaded successfully!")
print(f"   Parameters: {model.config.num_parameters / 1e6:.1f}M")

## Step 4: Add LoRA Adapters

In [ ]:
# LoRA Configuration
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=3407,
)

print("✅ LoRA adapters added!")

## Step 5: Load and Prepare Dataset (Alpaca)

In [ ]:
# Alpaca prompt template
ALPACA_PROMPT = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = ALPACA_PROMPT.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

print("📚 Loading Alpaca dataset...")
dataset = load_dataset("tatsu-lab/alpaca", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

# Split into train and eval
dataset_dict = dataset.train_test_split(test_size=0.05)
train_dataset = dataset_dict["train"]
eval_dataset = dataset_dict["test"]

print(f"✅ Dataset prepared!")
print(f"   Training: {len(train_dataset)} examples")
print(f"   Evaluation: {len(eval_dataset)} examples")

## Step 6: Configure Training

In [ ]:
# Training arguments
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    packing=True,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=100,  # Adjust for more training
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        evaluation_strategy="steps",
        eval_steps=10,
        save_steps=50,
        logging_steps=10,
        report_to="none"
    ),
)

print("✅ Training configuration ready!")

## Step 7: Start Fine-Tuning! 🚀

In [ ]:
print("🚀 Starting fine-tuning...")
print("⏱️  This will take 30-90 minutes depending on max_steps")
print("📊 Watch the loss decrease over time!\n")

trainer.train()

print("\n✅ Fine-tuning complete!")

## Step 8: Save Your Rudushi Model

In [ ]:
OUTPUT_DIR = "fine_tuned_rudushi"

print(f"💾 Saving model to {OUTPUT_DIR}...")
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("✅ Model saved successfully!")
print(f"📁 Location: /content/{OUTPUT_DIR}")

## Step 9: Test Your Rudushi Model

In [ ]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model)

test_prompt = ALPACA_PROMPT.format(
    "Write a Python function to reverse a string.",
    "",
    ""
)

inputs = tokenizer([test_prompt], return_tensors="pt").to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    use_cache=True,
    temperature=0.7,
    top_p=0.9
)

response = tokenizer.batch_decode(outputs)[0]
print("🧪 Test output:")
print(response)
print("\n✅ Test complete!")

## Step 10: Download Your Model

In [ ]:
from google.colab import files

print("📦 Creating model archive...")
!zip -r rudushi_model.zip fine_tuned_rudushi/

print("\n⬇️  Download your model:")
files.download("rudushi_model.zip")

print("\n✅ Model ready for download!")

## 🎉 Next Steps: Upload to Hugging Face

1. **Upload the model to your device:**
   - Transfer the downloaded `rudushi_model.zip` to your Android device
   - Extract it in `/data/data/com.termux/files/home/rudushi/`

2. **Convert to GGUF format:**
   ```bash
   cd /data/data/com.termux/files/home/rudushi
   python3 02_convert_to_gguf.py
   ```

3. **Upload to Hugging Face:**
   ```bash
   python3 03_upload_to_hf.py --token YOUR_HF_TOKEN
   ```

4. **Deploy in Termux:**
   ```bash
   ./rudushi
   # Then select: Chat with Rudushi
   ```

## 📚 Your Custom Rudushi Model

**Model Details:**
- Base: TinyLlama-1.1B
- Fine-tuned on: Alpaca dataset (52K instructions)
- Method: LoRA (Rank 16)
- Steps: 100 (adjustable)
- Upload to: megharudushi/Rudushi

Your Rudushi is ready to chat! 🤖✨